In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import random
from scipy import stats
from matplotlib.animation import FuncAnimation

## `make_polymer` Function for generating initial polymer
Just the function to make sure no overlap happens on the polymer.

In [2]:
def make_polymer(nbeads, dt=0.1) -> np.ndarray:
    polymer = np.zeros((nbeads + 1, 2), dtype=float) # makes the postion matrix for polymer
    polymer[:,0] = (polymer[:,0] + dt).cumsum()
            
    # return polymer
    return polymer

def check_overlap(xval, yval, tol=5e-2) -> bool:
    # check for overlap
    check = True
    for i, (x, y) in enumerate(zip(xval, yval)):
        xcond = np.any(np.abs(xval[:i] - x) <= tol) 
        ycond = np.any(np.abs(yval[:i] - y) <= tol)
        if np.any(xcond & ycond):
            check = False
    return check

# Applying the Active Brownian Model

Just as a reminder, the simplest update method for ABP is
\begin{align}
d\theta/dt &= \sqrt{2 D_R} W_{\theta} \\
dx/dt &= v \cos\theta(t) + \sqrt{2 D_T} W_x  \\
dy/dt &= v \sin\theta(t) + \sqrt{2 D_T} W_y
\end{align}

Which in coding terms becomes this:
\begin{align}
\theta_{i+1} &= \theta_{i} + \sqrt{2 D_R dt} w_{\theta,i} \\
x_{i+1} &=  x_{i} + v\cos \theta(t) dt + \sqrt{2 D_T dt} w_{x, i}  \\
y_{i+1} &=  y_{i} + v\sin(\theta(t)) dt + \sqrt{2 D_T dt} w_{y,i}
\end{align}

In [17]:
# ABP model parameters: 
# ballistic velocity,
# time step,
# rotational diffusion constant, 
# translational diffusion constant
vel = 2.0; dt = 0.01; Drot = 1.0; Dtrans = 0.3

#  ABP model
n_beads = 21
polymer = make_polymer(n_beads)
xvec = np.array([polymer[:, 0]])
yvec = np.array([polymer[:, 1]])
thetavec = random.uniform(-2*np.pi, 2 * np.pi, size=n_beads + 1).reshape(1, -1)

# start with a short walk of 100 steps and an
# initial x, y, theta coordinates for the polymer
x=xvec[0]; y = yvec[0]; theta = thetavec[0]
num_steps = 100

for i in range(num_steps):
    # calculate diffusive/random steps. For the x- and y-,we generate 
    # a random number between -1 & 1 to ensure that the walker can step in 
    # both directions(up/down and left/right).
    dx = vel*dt*np.cos(theta) + np.sqrt(2*Dtrans*dt)*random.normal(0, 1/3, size=n_beads + 1)
    dy = vel*dt*np.sin(theta) + np.sqrt(2*Dtrans*dt)*random.normal(0, 1/3, size=n_beads + 1)
    dtheta = np.sqrt(2*Drot*dt)*random.normal(0, np.pi, size=n_beads + 1)

    # update coordinates (including ballistic step)
    x += dx
    y += dy
    theta += dtheta

    # store successive positions in arrays
    xvec = np.vstack((xvec, x))
    yvec = np.vstack((yvec, y))
    thetavec = np.vstack((thetavec, theta))
    


#print(xvec.shape, yvec.shape, thetavec.shape)

## Plotting the Model via `Animation`

In [13]:
from IPython.display import HTML

# make your plot
fig, ax = plt.subplots(figsize=(6, 6))
ln, = ax.plot([], [], 'red', linewidth=3)

# get the limits
# this is to ensure the polymer
# fits in the box
lim = np.max([
    np.abs(xvec).max(), # get the biggest x term
    np.abs(yvec).max()  # get the biggest y term
])

def init():
    # set graph limits
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    return ln,

def update(frame):
    # now just update the line
    ln.set_data(xvec[frame], yvec[frame])
    return ln,

ani = FuncAnimation(fig, update, frames=np.arange(num_steps),
                    init_func=init, blit=True, repeat=True, interval=50)
# run the video
plt.close()
HTML(ani.to_html5_video())

Polymer above has the initial polymer with no overlap, but the beads all move independently of each other and that causes the polymer to "explode"

Next we wanted to make the polymer move in a linear fashion instead of having each polymer bead move independently of each other.  To do this we have the the polymer coordinates update in a sequential way so that the first bead is leading the whole polymer

In [24]:
# ABP model parameters: 
# ballistic velocity,
# time step,
# rotational diffusion constant, 
# translational diffusion constant
vel = 2.0; dt = 0.01; Drot = 1.0; Dtrans = 0.3

#  ABP model
n_beads = 21
polymer = make_polymer(n_beads)
xvec = np.array([polymer[:, 0]])
yvec = np.array([polymer[:, 1]])
thetavec = random.uniform(-2*np.pi, 2 * np.pi, size=n_beads + 1).reshape(1, -1)

# start with a short walk of 100 steps and an
# initial x, y, theta coordinates for the polymer
x=xvec[0]; y = yvec[0]; theta = thetavec[0]
num_steps = 100

for i in range(num_steps):
    # calculate diffusive/random steps. For the x- and y-,we generate 
    # a random number between -1 & 1 to ensure that the walker can step in 
    # both directions(up/down and left/right).
    dx = vel*dt*np.cos(theta[0]) + np.sqrt(2*Dtrans*dt)*random.normal(0, 1/3, size=1)
    dy = vel*dt*np.sin(theta[0]) + np.sqrt(2*Dtrans*dt)*random.normal(0, 1/3, size=1)
    dtheta = np.sqrt(2*Drot*dt)*random.normal(0, np.pi, size=1)

    # update coordinates (including ballistic step)
    x[1:] = x[:-1]
    y[1:] = y[:-1]
    theta[1:] = theta[:-1]
    x[0] += dx
    y[0] += dy
    theta[0] += dtheta

    # store successive positions in arrays
    xvec = np.vstack((xvec, x))
    yvec = np.vstack((yvec, y))
    thetavec = np.vstack((thetavec, theta))
    


#print(xvec.shape, yvec.shape, thetavec.shape)

In [25]:
from IPython.display import HTML

# make your plot
fig, ax = plt.subplots(figsize=(6, 6))
ln, = ax.plot([], [], 'red', linewidth=3)

# get the limits
# this is to ensure the polymer
# fits in the box
lim = np.max([
    np.abs(xvec).max(), # get the biggest x term
    np.abs(yvec).max()  # get the biggest y term
])

def init():
    # set graph limits
    ax.set_xlim(-lim, lim)
    ax.set_ylim(-lim, lim)
    return ln,

def update(frame):
    # now just update the line
    ln.set_data(xvec[frame], yvec[frame])
    return ln,

ani = FuncAnimation(fig, update, frames=np.arange(num_steps),
                    init_func=init, blit=True, repeat=True, interval=50)
# run the video
plt.close()
HTML(ani.to_html5_video())

Polymer moves in a sequential pattern and behaves more like a polymer with active brownian motion. This is in contrast to the simulations with LAMMPS where the polymers would simply diffuse as a whole in any given direction with no directionality to guide the polymer. This shows that with an initial "kick" the polymer will move in a specified direction.

Future work that could be done is to introduce terms that prevent polymer overlap as it moves and another term to keep the beads a specified distance apart from each other. This, however, could be used as an example to demostrate how a polymer could move with active Brownian motion.